In [ ]:
! Date

# 2 photon stimulation analysis


1. Define the path to the folders containing the data below. 
2. DO NOT CLICK "RUN ALL". Run each cell (by clicking "shift+enter") until you hit a "STOP" cell.
    - First STOP cell: Check whether there is a tail in the Thorsync frames (code will plot frames over time below), and define tail clean-up accordingly. Then continue running the notebook until the next STOP cell.
    - Subsequent STOP cells: Draw the ROIs (and later the background areas to normalize against), then continue running the notebook. 

Written by Laura Luebbert  
Last updated: Feb 28 17:38:52 PST 2022

### Define the directory containing the tif files (saved by ThorImage software):

In [ ]:
data_dir =  "path/to/thorimage_folder"

### Define the directory containing the sync files (saved by ThorSync software):

In [ ]:
sync_dir = "path/to/sync_folder"

### Define directory for pickled dictionary and csv file to be saved to:

In [ ]:
saving_dir = "path/to/output_folder"

Now you can start running the notebook until you find a "STOP" cell.

___

___

# Import packages

In [ ]:
# %load_ext blackcellmagic
%config InlineBackend.figure_format = 'retina'

In [ ]:
# Tools to read in the image files and filenames
import glob
import os
import re 

# Calculation and data frame tools
import numpy as np
import pandas as pd

# Image processing tools
import skimage
import skimage.io
import skimage.morphology

import bebi103

# Tools to create new folders
from pathlib import Path

# Tools to save dictionaries
import pickle

# To make a copy of a dictionary
import copy

# Load hdf5 and xml files into Python
import h5py
import xml.etree.ElementTree as ET

# Plotting tools
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.animation as animation

import bokeh
import bokeh_catplot
from bokeh.plotting import output_file, save
bokeh.io.output_notebook()

___

# Load in the data

In [ ]:
# Glob string for images (loads all .tif files)
im_glob = os.path.join(data_dir, "Chan*.tif")
im_glob_preview = os.path.join(data_dir, "*_Preview.tif")
im_glob_chanB = os.path.join(data_dir, "ChanB_*.tif")

# Get list of tif files in data directory (except for the "Preview" tif files and Channel B files - for now)
im_list = sorted([i for i in glob.glob(im_glob) if i not in glob.glob(im_glob_preview) and i not in glob.glob(im_glob_chanB)])

# Let's look at the first 10 entries
im_list[:10]

Create a nested dictionary with information about each sample coupled to the z-stack image matrix, as well as space for matrices added later:

In [ ]:
dicts = {}

for i in range(len(im_list)):
    # Get channel
    channel = im_list[i].split("/")[-1].split("_")[0]
    
    # Make sure we do not overwrite a previous dictionary entry    
    if dicts.get(channel) == None:
        dicts[channel] = {}
    
    # Get sample number
    sample = "_".join(im_list[i].split("/")[-1].split("_")[1:-1])
    
    # Make sure we do not overwrite a previous dictionary entry
    if dicts.get(channel, {}).get(sample) == None:
        
        dicts[channel][sample] = {        
            "matrix_orig": [],       # Original image z-stack matrices
            "df-f0": [],             # Image z-stack matrices showinf df/f0 for each pixel
            "f0_matrix": [],         # Matrix with f0 value of each individual pixel
            "matrix_sum": [],        # Image matrix of sum projection image 
            "df-f0_sum": [],         # df/f0 of each pixel in sum projection
            "filename": [],          # What the max projection will be named when saved locally
            "time_points": [],       # time points in seconds based on fps (counting up from 0)
            "clicks": [],            # The coordinates of "clicks" defining the ROI 
            "rois": [],              # ROI clicks converted to an ROI (polygon) 
            "roi_mask": [],          # Boolean matrix (mask) in the size of my original image with "True" values where the ROI is
            "clicks_bkg": [],        # The coordinates of "clicks" defining the background area to normalize against 
            "rois_bkg": [],          # bkg area clicks converted to a polygon
            "mean_int": [],          # Mean pixel value in ROI (not normalized)
            "mean_int_bkg": [],      # Mean pixel value in bkg area
            "mean_int_norm":[],      # Mean pixel value in ROI - mean pixel value in bkg area
#             "median_int": [],        # Median pixel value in ROI
#             "median_int_bkg": [],    # Median pixel value in bkg area
#             "median_int_norm": [],   # Median pixel value in ROI - median pixel value in bkg area
#             "raw_int_den": [],       # Raw integrated density of ROI (sum of all bkg normalized pixel values inside the ROI) 
#             "int_den": [],           # Integrated density of ROI (product of ROI area and mean bkg normalized pixel value inside the ROI)
#             "mean_fold_change": [],  # (Mean pixel value of ROI (not normalized) - Mean bkg pixel value) / Mean bkg pixel value
#             "median_fold_change": [] # (Median pixel value of ROI (not normalized) - Median bkg pixel value) / Median bkg pixel value
            }

    # Append original image matrices that belong together to form a z-stack
    dicts[channel][sample]["matrix_orig"].append(skimage.io.imread(im_list[i],is_ome=False))
    
    # Populate dictionary with original filename (without information automatically added by microscope)
    dicts[channel][sample]["filename"] = channel + "_" + sample

___

# Extract stimulation data from experiment files

Extract sample rate from xml file in sync folder:

In [ ]:
# Glob string for xml file in sync folder (loads only ThorRealTimeDataSettings file)
rtd_glob = os.path.join(sync_dir, "ThorRealTimeDataSettings.xml")

rtd_file = glob.glob(rtd_glob)

Define function to extract sample rate from ThorRealTimeDataSettings.xml file (written by Peter Weir):

In [ ]:
def get_sample_rate(inFileName):
    """Finds sample rate for a ThorSync .xml file
    usage:
    sampleRateHz = parse_thor_xml.get_sample_rate(inFileName)

    PTW 2015-08-07"""
    tree = ET.parse(inFileName)
    root = tree.getroot()
    for child in root:
        if child.tag == 'DaqDevices':
            for grandchild in child:
                if grandchild.tag == 'AcquireBoard' and grandchild.attrib['active'] == '1':
                    for greatgrandchild in grandchild:
                        if greatgrandchild.tag == 'SampleRate' and greatgrandchild.attrib['enable'] == '1':
                            sampleRateHz = float(greatgrandchild.attrib['rate'])
                            break

    return sampleRateHz

In [ ]:
sampleRateHz = get_sample_rate(rtd_file[0])

In [ ]:
sampleRateHz

Get hdf5 file from sync folder:

In [ ]:
# Glob string for hdf5 files in sync folder (loads only Episode001 file)
sync_glob = os.path.join(sync_dir, "Episode001.h5")

sync_file = glob.glob(sync_glob)
sync_file

In [ ]:
sync = h5py.File(sync_file[0], 'r')

In [ ]:
# Hdf5 files behave like dictionaries. List the keys in this hdf5 file:
list(sync.keys())

In [ ]:
# List keys inside "DI"
list(sync['DI'].keys())

Extract FrameOut to find time axis in seconds:

In [ ]:
frames = sync['DI']['FrameOut'][:].squeeze()

Compute time in seconds from "dirty" frames:

In [ ]:
timeSec = np.arange(len(frames))/sampleRateHz

Plot frames over time before cleanup:

In [ ]:
x = timeSec
y = frames

color = "black"

fig, ax = plt.subplots(figsize=(20, 7))

plt.plot(x, y, c=color)

# Define figure title
ax.set_title('Frames over time', weight='bold', size=17)

# Set axis labels
fontsize = 13
fontweight = 'normal'
fontproperties = {'weight':fontweight, 'size':fontsize}
ax.set_xlabel('Time (s)', fontproperties)
ax.set_ylabel('FrameOut', fontproperties)

## STOP - Based on the graph above, define whether or not there is a start and end tail of 0s frames in the cell below:

In [ ]:
# Define whether or not there is a tail
start_tail = True
end_tail = True

Clean up frames by removing 0s at the start and end of the recording if there is a tail:

In [ ]:
if start_tail == True:
    # Find first occurence of non-zero number in frames
    for i, value in enumerate(frames):
        if (value != 0):
            start_idx = i
            break
else:
    # If not tail at start, use first index so nothing is removed from start
    start_idx = 0

In [ ]:
if end_tail == True:        
    # Same on reversed array to find first occurence of non-zero number from end         
    for i, value in enumerate(frames[::-1]):
        if (value != 0):
            end_idx = i
            break
        
else:
    # If not tail at end, set to 1 (will be -1) so nothing is removed from end
    end_idx = 1

In [ ]:
# Slice out 0s at beginning and end
frames = frames[start_idx:-end_idx]

Compute time in seconds from clean frames:

In [ ]:
timeSec = np.arange(len(frames))/sampleRateHz

Plot frames over time after cleanup:

In [ ]:
x = timeSec
y = frames

color = "black"

fig, ax = plt.subplots(figsize=(20, 7))

plt.plot(x, y, c=color)

# Define figure title
ax.set_title('Frames over time after tail cleanup', weight='bold', size=17)

# Set axis labels
fontsize = 13
fontweight = 'normal'
fontproperties = {'weight':fontweight, 'size':fontsize}
ax.set_xlabel('Time (s)', fontproperties)
ax.set_ylabel('FrameOut', fontproperties)

Extract BleachOut channel data (channel used to save LED stimulus):

In [ ]:
led = sync['DI']['SignalGenerator'][:].squeeze()
# Slice out parts without frames
led = led[start_idx:-end_idx]

Plot LED stimulation trace over time (in seconds):

In [ ]:
x = timeSec
y = led

color = "black"

fig, ax = plt.subplots(figsize=(20, 7))

plt.plot(x, y, c=color)

# Define figure title
ax.set_title('LED stimulation trace', weight='bold', size=17)

# Set axis labels
fontsize = 13
fontweight = 'normal'
fontproperties = {'weight':fontweight, 'size':fontsize}
ax.set_xlabel('Time (s)', fontproperties)
ax.set_ylabel('LED', fontproperties)

Find time points where the LED turned on and off:

In [ ]:
# Convert unsigned interger lists to signed
# !!! Delete this cell for use with HQ!
led = led.astype(int)
timeSec = timeSec.astype(float)

In [ ]:
# Find time points when the next minus the previous value is > or < 0, and append to led_on and led_off, respectively
led_on = []
led_off = []

for idx, value in enumerate(led[:-1]):
    if led[idx+1] - value > 0:
        led_on.append(timeSec[idx])
        
    if led[idx+1] - value < 0:
        led_off.append(timeSec[idx])

In [ ]:
led_on

In [ ]:
led_off

Calculate frame rate (by dividing the number of frames in the last saved sample by duration of the sync file):

In [ ]:
fps = len(dicts[channel][sample]["matrix_orig"])/timeSec.max()
fps

In [ ]:
for k1_channel, v1_sample in dicts.items():
    for k2_sample, imagedata in v1_sample.items():
        # Populate dictionary with time points
        t = np.arange(len(imagedata["matrix_orig"])) / fps

        imagedata["time_points"] = t    

___
# Compute Sum projection for ROI drawing

Compute sum of frames for each image:

In [ ]:
for k1_channel, v1_sample in dicts.items():
    for k2_sample, imagedata in v1_sample.items():
        # Convert dict entry to array
        image = np.array(imagedata["matrix_orig"])

        # Sum z-project using numpy by definining the axis over which to sum the elements.
        summed_image = image.sum(axis=0)

        # Linearly scale down to 16-bit.
        summed_image = (summed_image/summed_image.max())*65535

        # Save summed image to dictionary
        imagedata["matrix_sum"] = summed_image
        
        # Save sum projections
        skimage.io.imsave(
            "{}/{}_sum_projection_{}.png".format(saving_dir, data_dir.split("/")[-1], imagedata["filename"]),
            summed_image,
            plugin=None,
            check_contrast=False,
        )

# Draw ROIs

To increase contrast, adjust the minimum and maximum intensity values of the displayed image. E.g. to increase brightness, lower the maximum intensity value (max_int_value) by defining a number or by dividing the 'dicts["ChanA"]["001_001_001"]["matrix_sum"].max()' value by 2, or 3, etc.

In [ ]:
# Define min and max intensity for image display
min_int_value = 0
max_int_value = dicts["ChanA"]["001_001_001"]["matrix_sum"].max()

In [ ]:
for k1_channel, v1_sample in dicts.items():
    for k2_sample, imagedata in v1_sample.items():
        # Enhance contrast using skimage.exposure.rescale_intensity function to define the min and max intensity values
        image = skimage.exposure.rescale_intensity(imagedata["matrix_sum"], in_range=(min_int_value, max_int_value))
        temp = bebi103.image.record_clicks(
            image,
            frame_height=800,
            title=imagedata["filename"],
            flip=False
        )
        # Save clicks to dictionary
        imagedata["clicks"] = temp

# STOP

Convert clicks to a tidy data frame:

In [ ]:
rois_for_polygon = []

for k1_channel, v1_sample in dicts.items():
    for k2_sample, imagedata in v1_sample.items():
        temp = imagedata["clicks"].to_df()

        # Add "roi" column (in this case there is just one ROI per sample with number "0")
        temp["roi"] = 0

        # Save clicks to dictionary as tidy data frame (this will overwrite the previously saved version of the clicks!)
        imagedata["clicks"] = temp
        
        # Save temp for plotting of ROI in heatmap animation
        rois_for_polygon.append(temp)

Use the bebi103.image.verts_to_roi function to convert the set of vertices (clicks) that define a polygon to a region of interest (the inside of the polygon):

In [ ]:
for k1_channel, v1_sample in dicts.items():
    for k2_sample, imagedata in v1_sample.items():
        imagedata["rois"] = [bebi103.image.verts_to_roi(imagedata["clicks"][['x', 'y']].values, 
                                    imagedata["matrix_sum"].shape[0], 
                                    imagedata["matrix_sum"].shape[1])
        for _, g in imagedata["clicks"].groupby('roi')]

Check that the ROIs are correct:

In [ ]:
plots = []

for k1_channel, v1_sample in dicts.items():
    for k2_sample, imagedata in v1_sample.items():
        # The function above returned 3 representations of our ROI:
        # roi = Boolean matrix (mask) in the size of my original image with "True" values where the ROI is
        # roi_bbox = Bounding box around ROI
        # roi_box = Boolean matrix in the size of the bounding box (roi_bbox) with "True" values where the ROI is
        roi, roi_bbox, roi_box = imagedata["rois"][0]
        
        imagedata["roi_mask"] = roi

        # Make grayscale image that is now RGB/CMY
        im = np.dstack(3 * [skimage.img_as_float(imagedata["matrix_sum"])])

        # Max out first channel
        im[roi, 0] = im.max()
        plots.append(
            bebi103.image.imshow(
                im,
                title="{}_roi".format(imagedata["filename"]),
                frame_height=250,
                cmap="rgb",
                flip=False,
            )
        )

# Look at the images
bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=3))

# Save images
for idx, plot in enumerate(plots):
    bokeh.io.export_png(plot, 
               filename="{}/{}_rois_{}.png".format(saving_dir, data_dir.split("/")[-1], idx),
               width=25000, height=25000)
#     export_svgs(plot, 
#                 filename="{}/{}_rois_{}.svg".format(saving_dir, data_dir.split("/")[-1], idx))

Calculate mean and median fluorescence value in ROI (not normalized against bkg):

In [ ]:
for k1_channel, v1_sample in dicts.items():
    for k2_sample, imagedata in v1_sample.items():
        
        roi, roi_bbox, roi_box = imagedata["rois"][0]
        im = imagedata["matrix_orig"]
        
        for stack in im:
            # Calculate mean and median intensity inside ROI for each individual stack
            mean_int = stack[roi].mean()
#             median_int = np.median(stack[roi])

            # Append to dictionary
            imagedata["mean_int"].append(mean_int)
#             imagedata["median_int"].append(median_int)

# Define background area

Define a small square outside of the ROI which is representative of the background fluorescence.

To increase contrast, adjust the minimum and maximum intensity values of the displayed image. E.g. to increase brightness, lower the maximum intensity value (max_int_value) by defining a number or by dividing the 'dicts["ChanA"]["001_001_001"]["matrix_sum"].max()' value by 2, or 3, etc.

In [ ]:
# Define min and max intensity for image display
min_int_value = 0
max_int_value = dicts["ChanA"]["001_001_001"]["matrix_sum"].max()

In [ ]:
for k1_channel, v1_sample in dicts.items():
    for k2_sample, imagedata in v1_sample.items():
        # Enhance contrast using skimage.exposure.rescale_intensity function to define the min and max intensity values
        image = skimage.exposure.rescale_intensity(imagedata["matrix_sum"], in_range=(min_int_value, max_int_value))
        temp = bebi103.image.record_clicks(
            image,
            frame_height=800,
            title=imagedata["filename"],
            flip=False
        )
        # Save clicks to dictionary
        imagedata["clicks_bkg"] = temp

# STOP

Convert clicks to a tidy data frame:

In [ ]:
for k1_channel, v1_sample in dicts.items():
    for k2_sample, imagedata in v1_sample.items():
        
        temp = imagedata["clicks_bkg"].to_df()

        # Add "roi" column (in this case there is just one ROI per sample with number "0")
        temp["roi"] = 0

        # Save clicks to dictionary as tidy data frame (this will overwrite the previously saved version of the clicks!)
        imagedata["clicks_bkg"] = temp

Use the bebi103.image.verts_to_roi function to convert the set of vertices (clicks) that define a polygon to a region of interest (the inside of the polygon):

In [ ]:
for k1_channel, v1_sample in dicts.items():
    for k2_sample, imagedata in v1_sample.items():
        
        dicts[k1_channel][k2_sample]["rois_bkg"] = [bebi103.image.verts_to_roi(imagedata["clicks_bkg"][['x', 'y']].values, 
                                    imagedata["matrix_sum"].shape[0], 
                                    imagedata["matrix_sum"].shape[1])
        for _, g in imagedata["clicks_bkg"].groupby('roi')]

Check that the ROIs are correct:

In [ ]:
plots = []

for k1_channel, v1_sample in dicts.items():
    for k2_sample, imagedata in v1_sample.items():
        
        # The function above returned 3 representations of our ROI:
        # roi = Boolean matrix (mask) in the size of my original image with "True" values where the ROI is
        # roi_bbox = Bounding box around ROI
        # roi_box = Boolean matrix in the size of the bounding box (roi_bbox) with "True" values where the ROI is
        roi, roi_bbox, roi_box = imagedata["rois_bkg"][0]

        # Make grayscale image that is now RGB/CMY
        im = np.dstack(3 * [skimage.img_as_float(imagedata["matrix_sum"])])

        # Max out first channel
        im[roi, 0] = im.max()
        plots.append(
            bebi103.image.imshow(
                im,
                title="{}_bkg-area".format(imagedata["filename"]),
                frame_height=250,
                cmap="rgb",
                flip=False,
            )
        )
        
# Look at the images
bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=3))
        
# Save first image (this assumes that all bkg areas are the same)
bokeh.io.export_png(plots[0], 
           filename="{}/{}_bkg_area.png".format(saving_dir, data_dir.split("/")[-1]),
           width=2400, height=2400)

Calculate mean and median fluorescence values in background area for each stack:

In [ ]:
for k1_channel, v1_sample in dicts.items():
    for k2_sample, imagedata in v1_sample.items():
        
        roi, roi_bbox, roi_box = imagedata["rois_bkg"][0]
        im = imagedata["matrix_orig"]
        
        for stack in im:
            # Calculate mean and median intensity inside bkg area for each individual stack
            mean_int_bkg = stack[roi].mean()
#             median_int_bkg = np.median(stack[roi])

            # Append to dictionary
            imagedata["mean_int_bkg"].append(mean_int_bkg)
#             imagedata["median_int_bkg"].append(median_int_bkg)

___

# Compute background normalized fluorescence values in each stack

In [ ]:
for k1_channel, v1_sample in dicts.items():
    for k2_sample, imagedata in v1_sample.items():
        
        roi, roi_bbox, roi_box = imagedata["rois_bkg"][0]
        im = imagedata["matrix_orig"]
            
        for i, stack in enumerate(im):
            
            # Get mean and median pixel values of ROI and bkg fo stack
            int_mean = imagedata["mean_int"][i]
            bkg_mean = imagedata["mean_int_bkg"][i]

            # Compute BG substracted mean and median pixel values in ROI for each stack
            norm_int_mean = int_mean - bkg_mean
            if norm_int_mean < 0:
                imagedata["mean_int_norm"].append(0)
            else: 
                imagedata["mean_int_norm"].append(norm_int_mean)

___

# Create a data frame and csv file for comparison of pixel values across conditions

In [ ]:
df = pd.DataFrame()

for k1_channel, v1_sample in dicts.items():
    for k2_sample, imagedata in v1_sample.items():
        x = 1

        for i, stack in enumerate(imagedata["matrix_orig"]):
            df = df.append(
                {"Filename" : imagedata["filename"],
                 "Channel" : k1_channel,
                 "Sample" : k2_sample,
                 "Time": imagedata["time_points"][i],
                 "Frame": x,
                 "Channel + Sample" : (k1_channel + " (" + k2_sample + ")"),
                 "Mean pixel value in ROI" : imagedata["mean_int"][i],
                 "Mean pixel value in bkg area" : imagedata["mean_int_bkg"][i],
                 "Norm_mean_pixel_value_in_ROI" : imagedata["mean_int_norm"][i],
                }, 
                ignore_index=True)
            x+=1

In [ ]:
# Display top of data frame
df.head()

___

# Plot raw mean fluorescence values over time

In [ ]:
fig, ax = plt.subplots(figsize=(20, 7))

for idx, sample in enumerate(np.unique(df["Channel + Sample"].values)):
    x = df[df["Channel + Sample"]==sample]["Time"].values
    y = df[df["Channel + Sample"]==sample]["Mean pixel value in ROI"].values

    ax.plot(x, y, c=cm.Set1(idx), label=sample)

# Add stimulation bar
for x1, x2 in zip(led_on, led_off):
    ax.axvspan(x1, x2, alpha=0.25, color='red')

# Set axis labels
fontsize = 13
fontweight = 'normal'
fontproperties = {'weight':fontweight, 'size':fontsize}
ax.set_xlabel('Time (s)', fontproperties)
ax.set_ylabel('Relative Fluorescence Units', fontproperties)

# Define figure title
ax.set_title('Raw Mean Fluorescence', weight='normal', size=fontsize+5)

# Add legend
ax.legend(fontsize=fontsize)

# Save figure
plt.savefig("{}/{}_mean-fluorescence.png".format(saving_dir, data_dir.split("/")[-1]),
            bbox_inches='tight', 
            dpi=300)

fig.show()

# Plot bkg normalized mean fluorescence values over time

In [ ]:
fig, ax = plt.subplots(figsize=(20, 7))

for idx, sample in enumerate(np.unique(df["Channel + Sample"].values)):
    x = df[df["Channel + Sample"]==sample]["Time"].values
    y = df[df["Channel + Sample"]==sample]["Norm_mean_pixel_value_in_ROI"].values

    ax.plot(x, y, c=cm.Set1(idx), label=sample)

# Add stimulation bar
for x1, x2 in zip(led_on, led_off):
    ax.axvspan(x1, x2, alpha=0.25, color='red')

# Set axis labels
fontsize = 13
fontweight = 'normal'
fontproperties = {'weight':fontweight, 'size':fontsize}
ax.set_xlabel('Time (s)', fontproperties)
ax.set_ylabel('Relative Fluorescence Units', fontproperties)

# Define figure title
ax.set_title('Background Normalized Mean Fluorescence', weight='normal', size=fontsize+5)

# Add legend
ax.legend(fontsize=fontsize)

# Save figure
plt.savefig("{}/{}_bkg-norm-mean-fluorescence.png".format(saving_dir, data_dir.split("/")[-1]),
            bbox_inches='tight', 
            dpi=300)

fig.show()

___

___

# Calculate $\Delta / F_0$ 

## Calculate $F_0$

Find all rows where time < time point that LED turns on:

In [ ]:
df_f0 = df.query("`Time` < @led_on[0]")

Calculate mean of bkg normalized pixel values inside ROI within first 5s and define as f0:

In [ ]:
f0 = np.mean(df_f0["Norm_mean_pixel_value_in_ROI"].values)
f0

Add column with $\Delta F / F_0$ values to original data frame

In [ ]:
df['df/f0'] = (df.Norm_mean_pixel_value_in_ROI - f0) / f0         

In [ ]:
df.head()

# Plot $\Delta F / F_0$ 

In [ ]:
fig, ax = plt.subplots(figsize=(20, 7))

for idx, sample in enumerate(np.unique(df["Channel + Sample"].values)):
    x = df[df["Channel + Sample"]==sample]["Time"].values
    y = df[df["Channel + Sample"]==sample]["df/f0"].values

    ax.plot(x, y, c=cm.Set1(idx), label=sample)

# Add stimulation bar
for x1, x2 in zip(led_on, led_off):
    ax.axvspan(x1, x2, alpha=0.25, color='red')

# Set axis labels
fontsize = 13
fontweight = 'normal'
fontproperties = {'weight':fontweight, 'size':fontsize}
ax.set_xlabel('Time (s)', fontproperties)
ax.set_ylabel('$\Delta F / F_0$', fontproperties)

# Define figure title
ax.set_title('$\Delta F / F_0$', weight='normal', size=fontsize+5)

# Add legend
ax.legend(fontsize=fontsize)

# Save figure
plt.savefig("{}/{}_df-f0.png".format(saving_dir, data_dir.split("/")[-1]),
            bbox_inches='tight', 
            dpi=300)

fig.show()

___

Add LED column to data frame:

In [ ]:
# Add column called "LED" to dataframe filled with 0s
df["LED"] = 0

for idx, timepoint in enumerate(df["Time"].values):
    for i, value in enumerate(led_on):
        if timepoint >= led_on[i] and timepoint < led_off[i]:
            df["LED"][idx] = 1     

# Save data frame to csv

In [ ]:
df.to_csv("{}/{}_2-photon_fluorescence_analysis.csv".format(saving_dir, data_dir.split("/")[-1]), index=False)

___

# Pickle dictionary containing images and clicks for later use
This dictionary contains the original iamge stacks, sum fluorescence images, names of the images, clicks as data frame, rois and the sum of the pixel intensity values inside the roi for each individual stack.   

In [ ]:
path = ("{}").format(saving_dir)

In [ ]:
# Use pickle to save dictionaries

# The advantage of HIGHEST_PROTOCOL is that files get smaller. This makes unpickling sometimes much faster.
with open(
    ("{}/{}_2-photon_fluorescence_analysis.pickle").format(path, data_dir.split("/")[-1]),
    "wb",
) as handle:
    pickle.dump(dicts, handle, protocol=pickle.HIGHEST_PROTOCOL)

___

# Plot heatmaps

In [ ]:
# Find number of pixels (assuming square image)
pixels = len(imagedata["matrix_orig"][0])

Find mean pixel value in backgrgound area for each frame:

In [ ]:
# Make another image stack where each image is of size 512,512 and filled with the average value on the background ROI for that frame
MEAN = [np.ones((pixels, pixels))*i for i in imagedata["mean_int_bkg"]]

In [ ]:
# X is our original z-stack
X = np.array(imagedata["matrix_orig"])

In [ ]:
# Y is the z-stack where each pixel value is subtracted by the mean pixel value in the background
Y = X-MEAN

In [ ]:
# Make all values that became < 0 during the background subtraction = 0
Y[Y<0] = 0

Find frames before LED turns on:

I define F0 as mean value of each pixel in frames 0 - five frames before LED turns on:

In [ ]:
if len(led_on) == 1: # LED turns on only once
    frame_on = np.array(np.argwhere(imagedata["time_points"]<led_on).reshape(-1))[-5]

else: # LED turns on more than once
    frame_on = np.array(np.argwhere(imagedata["time_points"]<led_on[0]).reshape(-1))[-5]

In [ ]:
F0 = Y[0:frame_on,:,:].mean(axis=0)

Calculate df/f0 z-stack:

In [ ]:
mtx = (Y-F0)/F0

Uncomment the two cells below to apply the ROI mask drawn above to see only this ROI in the heat map:

In [ ]:
# # Create array containing z copies of mask to match shape of Y
# mask = []

# for i in np.arange(len(Y)):
#     mask.append(imagedata["roi_mask"])

In [ ]:
# mtx[~np.array(mask)] = 0

## Plot sum projection of heat map

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

im = plt.imshow(mtx.sum(axis=0), origin='lower', cmap="gray")

#Set zoom
ax.set_xlim((0,pixels))
ax.set_ylim((0,pixels))

fontsize = 13

# Set axis labels
ax.set_xlabel('Pixel (x)', fontsize=fontsize)
ax.set_ylabel('Pixel (y)', fontsize=fontsize)

# Add heatmap legend
plt.colorbar(shrink=0.5).set_label('sum projection of $\Delta F / F_0$', fontsize=fontsize)

ax.set_title("$\Delta F / F_0$", weight='bold', fontsize=fontsize+5)

plt.savefig("{}/{}_df-f0_heatmap.png".format(saving_dir, data_dir.split("/")[-1]),
            bbox_inches='tight', 
            dpi=300)

fig.tight_layout()
plt.show()

## Plot animated heatmap

In [ ]:
%matplotlib tk
a = mtx

fig, ax = plt.subplots(figsize=(7,5))

frame = 0
idx = 0

# # For normalization of heatmap between thresholds:
# norm = matplotlib.colors.Normalize(vmin = 0, vmax = 2000, clip = False)
# im = plt.imshow(a[frame], origin='lower', cmap="hot", norm=norm)

im = plt.imshow(a[frame], origin='lower', cmap="gray")

# Set zoom
# ax.set_xlim((150,400))
# ax.set_ylim((150,400))

# Set axis labels
ax.set_xlabel('Pixel (x)')
ax.set_ylabel('Pixel (y)')

# Define figure title
# ax.set_title('$\Delta F / F_0$', weight='bold')

# Add heatmap legend
plt.colorbar(shrink=0.5).set_label('$\Delta F / F_0$')

# Define elapsed time text
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, color='white')

# Define size,location and color of circle showing stimulation
circle = plt.Circle((0.9, 0.9), 0.05, color='r', transform=ax.transAxes)
circle_status = False

# # Define polygon to show ROI
# points = rois_for_polygon[0].drop(['roi'], axis=1).values
# polygon = plt.Polygon(points, fill=None, edgecolor='white', linestyle=(0, (1,10)), linewidth=0.25)

def update(*args):
    global frame
    global idx 
    global circle_status

    im.set_array(a[frame])
    
    # Define when stimulation circle appears
    if df["LED"].values[idx] == 1:
        if circle_status == False:
            ax.add_artist(circle)
            circle_status = True
            print("Turned ON")
        else: 
            pass
        
    elif df["LED"].values[idx] == 0:
        if circle_status == False:
            pass
        else:
            circle.set_visible(False)
            print("Turned OFF")
            circle_status = False

    frame += 1
    frame %= len(a)
    
    time_text.set_text('Time in s: %.1f' % df["Time"].values[idx])
    idx += 1
        
    return im

# For display:
ani = animation.FuncAnimation(fig, update, interval=50)

# # For saving:
# ani = animation.FuncAnimation(fig, update, interval=200, save_count=len(a))

# # Set up formatting for saving
# Writer = animation.writers['ffmpeg']
# writer = Writer(fps=5, metadata=dict(artist='Me'), bitrate=1800)
# ani.save("{}/{}_df-f0_heatmap_movie.mp4".format(saving_dir, data_dir.split("/")[-1]), writer=writer)

plt.show()

___

In [ ]:
%load_ext watermark
%watermark -v -p numpy,pandas,skimage,bokeh,bebi103,jupyterlab